#  Data Exploration Keeling Curve 



 Development of readable maintainable code is not something often taught in an undergraduate Physics course. However, when working with others or on long projects, the veracity of 'Code is read more than it is written'-Guido Van Rossum becomes evident. Becuase of this, the author would recommend that you are accustomed to PEP-8 and PEP-484. In addition, the Google Python Style guide has some very prudent suggestions worth reading. This, in conjunction with good naming practices and reasonable amounts of comments, makes deciphering code a feasible task. Such practices outlined in these documents will help the reader and future contributors collaborate and maintain this and other work.

The initial author hopes that their original body of work is easily accessible and can be built off with relative ease and that his effort in making a framework for continuation bears fruit. 
